In [161]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [182]:
import pickle
import sys
import os
from pathlib import Path

# Добавляем путь на уровень выше
sys.path.append(str(Path(os.getcwd()).resolve().parent))

import numpy as np
import pandas as pd
import json
from pathlib import Path
from utils.feature_engineering import add_features
from utils.features import *

# Загрузка данных из np.memmap
def load_memmap(data_path, shape, dtype='float32', mode='r'):
    """Загружает данные из файла np.memmap."""
    return np.memmap(data_path, dtype=dtype, mode=mode, shape=shape)[:2016]

# Сохранение данных в np.memmap
def save_memmap(data, data_path, dtype='float32'):
    """Сохраняет данные в файл np.memmap."""
    # Преобразуем data_path в объект Path
    data_path = Path(data_path)
    
    # Проверяем, существует ли папка
    if not data_path.parent.exists():
        # Создаем папку, если она не существует
        data_path.parent.mkdir(parents=True, exist_ok=True)
        print(f"Папка {data_path.parent} создана.")

    # Сохраняем данные в np.memmap
    memmap = np.memmap(data_path, dtype=dtype, mode='w+', shape=data.shape)
    memmap[:] = data[:]
    memmap.flush()
    print(f"Данные сохранены в {data_path}.")

def load_pkl(pickle_file: str) -> object:
    """
    Load data from a pickle file.

    Args:
        pickle_file (str): Path to the pickle file.

    Returns:
        object: Loaded object from the pickle file.
    """

    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print(f'Unable to load data from {pickle_file}: {e}')
        raise
    return pickle_data

In [198]:
data_dir = Path('./data/all_data/PEMS-BAY/new2/')
data_path = data_dir / 'data.dat'
metadata_path = data_dir / 'desc.json'
adj_path = data_dir / 'adj_mx.pkl'

# Загрузка метаданных
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Загрузка данных
data_shape = (metadata['num_time_steps'], metadata['num_nodes'], metadata['num_features'])
data = load_memmap(data_path, shape=data_shape)

try:
    _, _, adj_mx_pb = load_pkl(adj_path)
except ValueError:
    adj_mx_pb = load_pkl(adj_path)

In [179]:
class convt(nn.Module):
    def __init__(self):
        super(convt, self).__init__()

    def forward(self, x, w):
        x = torch.einsum('bne, ek->bnk', (x, w))
        return x.contiguous()
    
class nconv(nn.Module):
    def __init__(self):
        super(nconv, self).__init__()

    def forward(self, x, A, dims):
        if dims == 2:
            x = torch.einsum('ncvl,vw->ncwl', (x, A))
        elif dims == 3:
            x = torch.einsum('ncvl,nvw->ncwl', (x, A))
        else:
            raise NotImplementedError('DFDGCN not implemented for A of dimension ' + str(dims))
        return x.contiguous()

class linear(nn.Module):
    """Linear layer."""

    def __init__(self, c_in, c_out):
        super(linear, self).__init__()
        self.mlp = torch.nn.Conv2d(c_in, c_out, kernel_size=(
            1, 1), padding=(0, 0), stride=(1, 1), bias=True)

    def forward(self, x):
        return self.mlp(x)

class gcn(nn.Module):
    """Graph convolution network."""

    def __init__(self, c_in, c_out, dropout, support_len=3, order=2):
        super(gcn, self).__init__()
        self.nconv = nconv()

        self.c_in = c_in
        c_in = (order * (support_len + 1) + 1) * self.c_in
        self.mlp = linear(c_in, c_out)
        self.dropout = dropout
        self.order = order

    def forward(self, x, support):

        out = [x]
        for a in support:
            x1 = self.nconv(x, a.to(x.device), a.dim())
            out.append(x1)

            for k in range(2, self.order + 1):
                x2 = self.nconv(x1, a.to(x1.device), a.dim())
                out.append(x2)
                x1 = x2
        h = torch.cat(out, dim=1)
        h = self.mlp(h)
        h = F.dropout(h, self.dropout, training=self.training)
        return h

def dy_mask_graph(adj, k):
    M = []
    for i in range(adj.size(0)):
        adp = adj[i]
        mask = torch.zeros( adj.size(1),adj.size(2)).to(adj.device)
        mask = mask.fill_(float("0"))
        s1, t1 = (adp + torch.rand_like(adp) * 0.01).topk(k, 1)
        mask = mask.scatter_(1, t1, s1.fill_(1))
        M.append(mask)
    mask = torch.stack(M,dim=0)
    adj = adj * mask
    return adj

def cat(x1,x2):
    M = []
    for i in range(x1.size(0)):
        x = x1[i]
        new_x = torch.cat([x,x2],dim=1)
        M.append(new_x)
    result = torch.stack(M,dim=0)
    return result


class DFDGCN(nn.Module):

    def __init__(self, num_nodes, dropout=0.3, supports=None,
                    gcn_bool=True, addaptadj=True, aptinit=None,
                    in_dim=2, out_dim=12, residual_channels=32,
                    dilation_channels=32, skip_channels=256, end_channels=512,
                    kernel_size=2, blocks=4, layers=2, a=1, seq_len=12, affine=True, fft_emb=10, identity_emb=10, hidden_emb=30, subgraph=20):
        super(DFDGCN, self).__init__()
        self.dropout = dropout
        self.blocks = blocks
        self.layers = layers
        self.gcn_bool = gcn_bool
        self.addaptadj = addaptadj
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.residual_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        self.bn = nn.ModuleList()
        self.gconv = nn.ModuleList()
        self.seq_len = seq_len
        self.a = a

        self.start_conv = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))

        self.supports = supports
        self.emb = fft_emb
        self.subgraph_size = subgraph
        self.identity_emb = identity_emb
        self.hidden_emb = hidden_emb
        self.fft_len = round(seq_len//2) + 1
        self.Ex1 = nn.Parameter(torch.randn(self.fft_len, self.emb), requires_grad=True)
        self.Wd = nn.Parameter(torch.randn(num_nodes,self.emb + self.identity_emb + self.seq_len * (in_dim-1), self.hidden_emb), requires_grad=True)
        self.Wxabs = nn.Parameter(torch.randn(self.hidden_emb, self.hidden_emb), requires_grad=True)

        self.mlp = linear(residual_channels * 4,residual_channels)
        self.layersnorm = torch.nn.LayerNorm(normalized_shape=[num_nodes,self.hidden_emb], eps=1e-08,elementwise_affine=affine)
        self.convt = convt()

        self.node1 = nn.Parameter(
            torch.randn(num_nodes, self.identity_emb), requires_grad=True)
        self.drop = nn.Dropout(p=dropout)

        self.T_i_D_emb = nn.Parameter(
            torch.empty(288, self.seq_len))
        self.D_i_W_emb = nn.Parameter(
            torch.empty(7, self.seq_len))
        self.G_emb = nn.Parameter(
            torch.empty(num_nodes, self.seq_len))

        receptive_field = 1
        self.reset_parameter()
        self.supports_len = 0
        if not addaptadj:
            self.supports_len -= 1
        if supports is not None:
            self.supports_len += len(supports)
        if gcn_bool and addaptadj:
            if aptinit is None:
                if supports is None:
                    self.supports = []
                self.nodevec1 = nn.Parameter(
                    torch.randn(num_nodes, self.emb), requires_grad=True)
                self.nodevec2 = nn.Parameter(
                    torch.randn(self.emb, num_nodes), requires_grad=True)
                self.supports_len += 1
            else:
                if supports is None:
                    self.supports = []
                m, p, n = torch.svd(aptinit)
                initemb1 = torch.mm(m[:, :10], torch.diag(p[:10] ** 0.5))
                initemb2 = torch.mm(torch.diag(p[:10] ** 0.5), n[:, :10].t())
                self.nodevec1 = nn.Parameter(initemb1, requires_grad=True)
                self.nodevec2 = nn.Parameter(initemb2, requires_grad=True)
                self.supports_len += 1

        for b in range(blocks):
            additional_scope = kernel_size - 1
            new_dilation = 1
            for i in range(layers):
                # dilated convolutions
                self.filter_convs.append(nn.Conv2d(in_channels=residual_channels,
                                                   out_channels=dilation_channels,
                                                   kernel_size=(1, kernel_size), dilation=new_dilation))

                self.gate_convs.append(nn.Conv2d(in_channels=residual_channels,
                                                 out_channels=dilation_channels,
                                                 kernel_size=(1, kernel_size), dilation=new_dilation))

                # 1x1 convolution for residual connection
                self.residual_convs.append(nn.Conv2d(in_channels=dilation_channels,
                                                     out_channels=residual_channels,
                                                     kernel_size=(1, 1)))

                # 1x1 convolution for skip connection
                self.skip_convs.append(nn.Conv2d(in_channels=dilation_channels,
                                                 out_channels=skip_channels,
                                                 kernel_size=(1, 1)))
                self.bn.append(nn.BatchNorm2d(residual_channels))
                new_dilation *= 2
                receptive_field += additional_scope
                additional_scope *= 2
                if self.gcn_bool:
                    self.gconv.append(
                        gcn(dilation_channels, residual_channels, dropout, support_len=self.supports_len))
        self.end_conv_1 = nn.Conv2d(in_channels=skip_channels,
                                    out_channels=end_channels,
                                    kernel_size=(1, 1),
                                    bias=True)

        self.end_conv_2 = nn.Conv2d(in_channels=end_channels,
                                    out_channels=out_dim,
                                    kernel_size=(1, 1),
                                    bias=True)

        self.receptive_field = receptive_field

    def reset_parameter(self):
        nn.init.xavier_uniform_(self.T_i_D_emb)
        nn.init.xavier_uniform_(self.D_i_W_emb)


    def forward(self, history_data: torch.Tensor) -> torch.Tensor:
        """Feedforward function of DFDGCN; Based on Graph WaveNet

        Args:
            history_data (torch.Tensor): shape [B, L, N, C]

        Graphs:
            predefined graphs: two graphs; [2, N, N] : Pre-given graph structure, including in-degree and out-degree graphs

            self-adaptive graph: [N, N] : Self-Adaptively constructed graphs with two learnable parameters
                torch.mm(self.nodevec1, self.nodevec2)
                    nodevec: [N, Emb]

            dynamic frequency domain graph: [B, N, N] : Data-driven graphs constructed with frequency domain information from traffic data
                traffic_data : [B, N, L]
                frequency domain information : [B, N, L/2.round + 1] ------Embedding ------[B, N, Emb2]
                Identity embedding : learnable parameter [N, Emb3]
                Time embedding : Week and Day : [N, 7] [N, 24(hour) * 12 (60min / 5min due to sampling)] ------Embedding ------ [N, 2 * Emb4]
                Concat frequency domain information + Identity embedding + Time embedding ------Embedding , Activating, Normalization and Dropout
                Conv1d to get adjacency matrix

        Returns:
            torch.Tensor: [B, L, N, 1]
        """
        #num_feat = model_args["num_feat"]
        input = history_data.transpose(1, 3).contiguous()[:,:,:,:]
        
        data = history_data

        in_len = input.size(3)
        if in_len < self.receptive_field:
            x = nn.functional.pad(
                input, (self.receptive_field-in_len, 0, 0, 0))
        else:
            x = input
        x = self.start_conv(x)

        skip = 0
        if self.gcn_bool and self.addaptadj and self.supports is not None:


            gwadp = F.softmax(
                F.relu(torch.mm(self.nodevec1, self.nodevec2)), dim=1)

            new_supports = self.supports + [gwadp] # pretrained graph in DCRNN and self-adaptive graph in GWNet

            # Construction of dynamic frequency domain graph
            xn1 = input[:, 0, :, -self.seq_len:]

            T_D = self.T_i_D_emb[(data[:, :, :, 1] * 288).type(torch.LongTensor)][:, -1, :, :]
            D_W = self.D_i_W_emb[(data[:, :, :, 2] * 7).type(torch.LongTensor)][:, -1, :, :]
            G_E = self.G_emb[(data[:, :, :, 3]).type(torch.LongTensor)][:, -1, :, :]

            xn1 = torch.fft.rfft(xn1, dim=-1)
            xn1 = torch.abs(xn1)

            xn1 = torch.nn.functional.normalize(xn1, p=2.0, dim=1, eps=1e-12, out=None)
            xn1 = torch.nn.functional.normalize(xn1, p=2.0, dim=2, eps=1e-12, out=None) * self.a

            xn1 = torch.matmul(xn1, self.Ex1)
            xn1k = cat(xn1, self.node1)
            x_n1 = torch.cat([xn1k, T_D, D_W,G_E], dim=2)
            x1 = torch.bmm(x_n1.permute(1,0,2),self.Wd).permute(1,0,2)
            x1 = torch.relu(x1)
            x1k = self.layersnorm(x1)
            x1k = self.drop(x1k)
            adp = self.convt(x1k, self.Wxabs)
            adj = torch.bmm(adp, x1.permute(0, 2, 1))
            adp = torch.relu(adj)
            adp = dy_mask_graph(adp, self.subgraph_size)
            adp = F.softmax(adp, dim=2)
            new_supports = new_supports + [adp]

        # WaveNet layers
        for i in range(self.blocks * self.layers):

            # dilated convolution
            residual = x
            filter = self.filter_convs[i](residual)
            filter = torch.tanh(filter)
            gate = self.gate_convs[i](residual)
            gate = torch.sigmoid(gate)
            x = filter * gate

            # parametrized skip connection
            s = x
            s = self.skip_convs[i](s)
            try:
                skip = skip[:, :, :,  -s.size(3):]
            except:
                skip = 0
            skip = s + skip

            if self.gcn_bool and self.supports is not None:
                if self.addaptadj:
                    x = self.gconv[i](x, new_supports)

                else:
                    x = self.gconv[i](x, self.supports)
            else:
                x = self.residual_convs[i](x)
            x = x + residual[:, :, :, -x.size(3):]

            x = self.bn[i](x)

        x = F.relu(skip)
        x = F.relu(self.end_conv_1(x))
        x = self.end_conv_2(x)
        return x

In [199]:
data = data[:, :, [0,1,2,6]]
data.shape

(2016, 325, 4)

In [200]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

# Данные и параметры
L, N, C = data.shape  # [2016, 325, C]
batch_size = 16
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2
seq_len = 12  # Количество временных шагов на вход
pred_len = 1  # Количество временных шагов для предсказания

# Индексы каналов для нормализации
normalize = True
channels_to_normalize = [0,3]  # Пример: нормализуем каналы 0 и 3

# Проверка корректности разделения данных
assert train_ratio + val_ratio + test_ratio == 1.0, "Сумма долей train, val и test должна быть равна 1.0"

# Разделение данных на train, val и test
num_samples = data.shape[0]  # Количество временных шагов (L)
train_size = int(num_samples * train_ratio)
val_size = int(num_samples * val_ratio)
test_size = num_samples - train_size - val_size

train_data = data[:train_size, :, :]  # [train_size, N, C]
val_data = data[train_size:train_size + val_size, :, :]  # [val_size, N, C]
test_data = data[train_size + val_size:, :, :]  # [test_size, N, C]

# Убедитесь, что данные имеют разрешение на запись
train_data = train_data.copy()
val_data = val_data.copy()
test_data = test_data.copy()

# Нормализация данных
if normalize:
    assert all(0 <= ch < C for ch in channels_to_normalize), "Индексы каналов выходят за пределы допустимого диапазона"
    channel_max = train_data[:, :, channels_to_normalize].max(axis=(0, 1), keepdims=True)  # Форма [1, 1, len(channels_to_normalize)]
    channel_max[channel_max == 0] = 1.0
    train_data[:, :, channels_to_normalize] = train_data[:, :, channels_to_normalize] / channel_max
    val_data[:, :, channels_to_normalize] = val_data[:, :, channels_to_normalize] / channel_max
    test_data[:, :, channels_to_normalize] = test_data[:, :, channels_to_normalize] / channel_max


# Создание кастомного Dataset
class TrafficDataset(Dataset):
    def __init__(self, data, seq_len, pred_len):
        super().__init__()
        self.data = data  # Форма [L, N, C]
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        # Количество возможных последовательностей
        return self.data.shape[0] - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        # Извлекаем последовательность входных данных
        x = self.data[idx:idx + self.seq_len, :, :]  # Форма [seq_len, N, C]
        # Извлекаем целевую последовательность
        y = self.data[idx + self.seq_len:idx + self.seq_len + self.pred_len, :, 0]  # Форма [pred_len, N, C]
        return x, y

# Создание DataLoader
train_dataset = TrafficDataset(train_data, seq_len, pred_len)
val_dataset = TrafficDataset(val_data, seq_len, pred_len)
test_dataset = TrafficDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [201]:
# Определение устройства
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'  # Устанавливаем CPU вместо CUDA
print(device)

# Обновление модели, данных и вычислений
supports = [torch.tensor(adj_mx_pb, dtype=torch.float32)]
model = DFDGCN(num_nodes=N, supports=supports, in_dim=C, out_dim=pred_len).to(device)  # Модель на CPU
criterion = nn.MSELoss()  # Функция потерь
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

cuda


In [202]:
from torch.utils.tensorboard import SummaryWriter
import time
# Инициализация TensorBoard
writer = SummaryWriter(log_dir=f"runs/traffic_prediction/clustering_coefficient")

def compute_metrics(output, target):
    """Вычисление метрик MAE, RMSE, MAPE."""
    abs_error = torch.abs(output - target).sum().item()
    mae = abs_error / len(target)
    rmse = ((output - target) ** 2).sum().item() / len(target)
    rmse = rmse ** 0.5
    mape = (abs_error / torch.abs(target).sum().item()) if torch.abs(target).sum().item() != 0 else 0
    return mae, rmse, mape

def train_val_test_model(model, train_loader, val_loader, test_loader, epochs):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        # === Тренировка ===
        model.train()
        train_loss, train_mae, train_rmse, train_mape = 0.0, 0.0, 0.0, 0.0
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)

        for x, y in train_loader_tqdm:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x).squeeze(-1)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)

            mae, rmse, mape = compute_metrics(output, y)
            train_mae += mae
            train_rmse += rmse
            train_mape += mape

        train_loss /= len(train_loader.dataset)
        train_mae /= len(train_loader)
        train_rmse /= len(train_loader)
        train_mape /= len(train_loader)

        writer.add_scalar("Loss/Train", train_loss, epoch + 1)
        writer.add_scalar("MAE/Train", train_mae, epoch + 1)
        writer.add_scalar("RMSE/Train", train_rmse, epoch + 1)
        writer.add_scalar("MAPE/Train", train_mape, epoch + 1)

        # === Валидация ===
        model.eval()
        val_loss, val_mae, val_rmse, val_mape = 0.0, 0.0, 0.0, 0.0
        val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation", leave=False)

        with torch.no_grad():
            for x, y in val_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                val_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                val_mae += mae
                val_rmse += rmse
                val_mape += mape

        val_loss /= len(val_loader.dataset)
        val_mae /= len(val_loader)
        val_rmse /= len(val_loader)
        val_mape /= len(val_loader)

        writer.add_scalar("Loss/Validation", val_loss, epoch + 1)
        writer.add_scalar("MAE/Validation", val_mae, epoch + 1)
        writer.add_scalar("RMSE/Validation", val_rmse, epoch + 1)
        writer.add_scalar("MAPE/Validation", val_mape, epoch + 1)

        # === Тестирование ===
        test_loss, test_mae, test_rmse, test_mape = 0.0, 0.0, 0.0, 0.0
        test_loader_tqdm = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs} - Testing", leave=False)

        with torch.no_grad():
            for x, y in test_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                test_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                test_mae += mae
                test_rmse += rmse
                test_mape += mape

        test_loss /= len(test_loader.dataset)
        test_mae /= len(test_loader)
        test_rmse /= len(test_loader)
        test_mape /= len(test_loader)

        writer.add_scalar("Loss/Test", test_loss, epoch + 1)
        writer.add_scalar("MAE/Test", test_mae, epoch + 1)
        writer.add_scalar("RMSE/Test", test_rmse, epoch + 1)
        writer.add_scalar("MAPE/Test", test_mape, epoch + 1)

        # Сохранение лучшей модели
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")

    writer.close()


In [203]:
train_val_test_model(model, train_loader, val_loader, test_loader, epochs=50)